In [82]:
df_train = pd.read_csv('../input/train.csv', index_col=0)
df_test = pd.read_csv('../input/test.csv', index_col=0)

In [83]:
X = df_train.drop(['SalePrice'], axis=1)
y = df_train['SalePrice']

In [84]:
df_train.dtypes.value_counts()

object     43
int64      34
float64     3
dtype: int64

In [85]:
all_data = pd.concat([X, df_test], axis=0)
all_data.shape

(2919, 79)

In [86]:
isnull_count = all_data.isnull().sum()
isnull_count = isnull_count[isnull_count > 0].sort_values(ascending=False)

In [87]:
left = pd.DataFrame(isnull_count,columns=['nums'])
right = pd.DataFrame(df_train.dtypes,columns=['type'])

In [88]:
left.join(right).head()

,nums,type
PoolQC,2909,object
MiscFeature,2814,object
Alley,2721,object
Fence,2348,object
FireplaceQu,1420,object


In [89]:
all_data.MSSubClass = all_data.MSSubClass.astype(str)

In [92]:
all_data.MSSubClass.dtypes

dtype('O')

In [93]:
all_dummy = pd.get_dummies(all_data)

In [98]:
all_dummy.fillna(all_dummy.mean(), inplace=True)

In [100]:
X_train = all_dummy.loc[df_train.index]
X_test = all_dummy.loc[df_test.index]

In [102]:
y = np.log1p(y)

In [103]:
from xgboost import XGBRegressor

xgbr = XGBRegressor(seed=666)

In [104]:
xgbr.fit(X_train, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=666,
       silent=True, subsample=1)

In [107]:
y_pred = xgbr.predict(X_test)
y_pred

array([11.7312975, 11.958404 , 12.099988 , ..., 12.019189 , 11.683053 ,
       12.362045 ], dtype=float32)

In [109]:
res = pd.DataFrame({'ID': df_test.index, 'SalePrice': np.expm1(y_pred)})
res.to_csv('xgb-base.csv', index=None)